In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'marathi-hasoc-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4573657%2F7809159%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240311%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240311T042435Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D77e4af1bc2daf32647a27a9073c4c7939e7f807620d82365860f5d138ea6040014c791eebeef3d3c5e77197a92ec8baebe90dff176323a50665e12f998443f6fc8cdee9b29ceec85e3ceed64d8e279552391cd9e6d0c5520c3596ab287d386702b5cad0a1f1284ca853cba2930ac99c06e7641e94745d5a5b0735574e4b62abe181aee785d5ab7f0fce9ab7cfc3fe1b7bff2c9735b8dae06920d0b1df82b458c3314d9d785c875aeef78014f543933ec2f86debe874cd5261cf39852090bde7907b14f95ba60492f183b6aea5c602a8db93001abf3c67c96df70a84250816b3513d58903489972e062962cc22b011ddb2681597246dd64ca5bf66254a299223e,marathi-molddataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4573667%2F7809170%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240311%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240311T042435Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D80d50473165319e7a703cfe6f4bba6b77d1403dd4f88e162cceedbe44a73740d3c340b2a531e61225b3328bf8a4eb7a30b37800de9f06ee6aec8e43741c72b20011c4a67aac980d0591656addbfc9d1c2815e95423cba23af0a0a1e584aa898642a63af27be712d6444b75fdbd20853fc10f2774ba25931fde976e87430870a55d5fbf52ed04aa6cd7e902bf0fbb4a6e6cc2c7db36cb376b5394bfae7505f3e3f8d92d9f8a17f0f662aeef0b9c7cd0e9c2f693f3baf7ae7c99d84743071faabfa4e311f3d2b98223802626f73522465daa9a84237e578ea82ab69cb3f6b6ab271bf70afbe949fa4b8ac023f990b8c4b2be4052816a8662d7a4562d0a5d31851f'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 138964 bytes downloaded
Downloaded and uncompressed: marathi-hasoc-dataset
[==================================================] 170941 bytes downloaded
Downloaded and uncompressed: marathi-molddataset
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/marathi-molddataset/MOLDV2_train.csv
/kaggle/input/marathi-hasoc-dataset/mr_Hasoc2021_train.xlsx


In [ ]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  # Show all columns
df = pd.read_csv('/kaggle/input/marathi-molddataset/MOLDV2_train.csv')

# Display the first few rows of the dataset
print(df.head())


                                                                                                                     tweet  \
0       आजच्या जनता दरबारात जळगाव जिल्ह्यातील चाळीसगावचे रहिवासी माजी सैनिक सोनू महाजन आले होते २०१६ साली भाजपचे तत्कालीन…   
1                     कुणी कविता करत असतं तर कुणी कविता जगत असतं कुणी कविता वाचत असतं तर कुणाला कविताच वाचवत असते © पल्लवी   
2        आम्हाला इतिहासातील औरंगजेबशी काही घेणे नाही आमच्या कडे आमचा बेकायदेशीर रित्या आलेला हक्काचा औरंगजेब आहे तो एका क…   
3                            गँभीर प्रकरण महाराष्ट्राची अवस्था बिकट आहे भाषणात मोठे शब्द वापरणे ऐकले कृती करावी उद्योग भी…   
4  कब्झा हा कन्नड चित्रपट लवकरच मराठी मध्ये डब्ब होऊन प्रदर्शित होणार आहे जर ह्या चित्रपटाला चांगला प्रतिसाद मिळाला तर आप…   

       subtask_a subtask_b subtask_c  
0  not offensive       NaN       NaN  
1  not offensive       NaN       NaN  
2  not offensive       NaN       NaN  
3  not offensive       NaN       NaN  
4  not offensive       NaN       NaN  


**Remove unwanted columns**

In [ ]:
# Specify the columns to remove
columns_to_remove = ['subtask_b', 'subtask_c']

# Remove columns that are in the specified list
df.drop(columns=columns_to_remove, inplace=True)


**Label Encoding**

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'task_1' column
df['subtask_a'] = label_encoder.fit_transform(df['subtask_a'])
print(df.head())  #Hate means 0

                                                                                                                     tweet  \
0       आजच्या जनता दरबारात जळगाव जिल्ह्यातील चाळीसगावचे रहिवासी माजी सैनिक सोनू महाजन आले होते २०१६ साली भाजपचे तत्कालीन…   
1                     कुणी कविता करत असतं तर कुणी कविता जगत असतं कुणी कविता वाचत असतं तर कुणाला कविताच वाचवत असते © पल्लवी   
2        आम्हाला इतिहासातील औरंगजेबशी काही घेणे नाही आमच्या कडे आमचा बेकायदेशीर रित्या आलेला हक्काचा औरंगजेब आहे तो एका क…   
3                            गँभीर प्रकरण महाराष्ट्राची अवस्था बिकट आहे भाषणात मोठे शब्द वापरणे ऐकले कृती करावी उद्योग भी…   
4  कब्झा हा कन्नड चित्रपट लवकरच मराठी मध्ये डब्ब होऊन प्रदर्शित होणार आहे जर ह्या चित्रपटाला चांगला प्रतिसाद मिळाला तर आप…   

   subtask_a  
0          1  
1          1  
2          1  
3          1  
4          1  


In [ ]:
print(df[df['subtask_a']==0].head())

                                                                                                                  tweet  \
1451                      राम कदम वागण्यात नाही तर बोलण्यात चुकला बीजेपी मग अजितदादा कुठे तुमच्या तोंडात मूतले होते का?   
1452                                     हीच का तुमची शिवसेने चि शिकवण आपली आई म्हणजे दैवत आणि दुसऱ्याची आई म्हणजे रांड   
1453                     हे वाचा गाढवांनो आणि हे ही सांगा की तुमच्या मॅडम आणि पप्पूने  साली काय करार केला चीनी कम्युनि…   
1454                                                                           भक्त आंधळे असतात मूर्खा ना काही कळत नाही   
1455  १९६६ साली छत्रपती शिवाजी महाराज या जागतिक दर्जाच्या सम्राटाचे नाव () वापरून ज्यांनी शिव सेना स्थापन केली त्यांची…   

      subtask_a  
1451          0  
1452          0  
1453          0  
1454          0  
1455          0  


In [ ]:
# Specify the index of the row you want to export
row_index = 254  # Change this to the desired row index

# Extract the content of the specified row
specific_row_content = df.at[row_index, 'tweet']

print(specific_row_content)


पाली हिलच्या कार्यालयानंतर कंगनाच्या फ्लॅटवर बीएमसीचा डोळा


**Drop null values**

In [ ]:
df = df.dropna(subset=['tweet'])

**Remove unwanted patterns in text**

In [ ]:
import re
#  remove "@USER"
df['tweet'] = df['tweet'].str.replace(r'@user', '', flags=re.IGNORECASE)

pattern = r'(\d+)\sसाली'
# remove above pattern which comes with date
df['tweet'] = df['tweet'].apply(lambda x: re.sub(pattern, '', x))

# remove special characters, numbers
df['tweet'] = df['tweet'].str.replace('[\d@?()~|‘•©-]', '', regex=True)

# remove more than one spaces
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\s+', ' ', x))

# Display the modified DataFrame
print(df['tweet'].head())



0                   आजच्या जनता दरबारात जळगाव जिल्ह्यातील चाळीसगावचे रहिवासी माजी सैनिक सोनू महाजन आले होते भाजपचे तत्कालीन…
1                         कुणी कविता करत असतं तर कुणी कविता जगत असतं कुणी कविता वाचत असतं तर कुणाला कविताच वाचवत असते पल्लवी
2          आम्हाला इतिहासातील औरंगजेबशी काही घेणे नाही आमच्या कडे आमचा बेकायदेशीर रित्या आलेला हक्काचा औरंगजेब आहे तो एका क…
3                              गँभीर प्रकरण महाराष्ट्राची अवस्था बिकट आहे भाषणात मोठे शब्द वापरणे ऐकले कृती करावी उद्योग भी…
4    कब्झा हा कन्नड चित्रपट लवकरच मराठी मध्ये डब्ब होऊन प्रदर्शित होणार आहे जर ह्या चित्रपटाला चांगला प्रतिसाद मिळाला तर आप…
Name: tweet, dtype: object


<ipython-input-9-0c02a56ed1e2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet'] = df['tweet'].str.replace(r'@user', '', flags=re.IGNORECASE)
<ipython-input-9-0c02a56ed1e2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet'] = df['tweet'].apply(lambda x: re.sub(pattern, '', x))


**Remove Punctuation marks**

In [ ]:
import string
# Function to remove punctuations from Marathi text
def remove_punctuations(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# Apply the function to the 'tweet' column
df['tweet'] = df['tweet'].apply(remove_punctuations)
print(df['tweet'].head())

0                   आजच्या जनता दरबारात जळगाव जिल्ह्यातील चाळीसगावचे रहिवासी माजी सैनिक सोनू महाजन आले होते भाजपचे तत्कालीन…
1                         कुणी कविता करत असतं तर कुणी कविता जगत असतं कुणी कविता वाचत असतं तर कुणाला कविताच वाचवत असते पल्लवी
2          आम्हाला इतिहासातील औरंगजेबशी काही घेणे नाही आमच्या कडे आमचा बेकायदेशीर रित्या आलेला हक्काचा औरंगजेब आहे तो एका क…
3                              गँभीर प्रकरण महाराष्ट्राची अवस्था बिकट आहे भाषणात मोठे शब्द वापरणे ऐकले कृती करावी उद्योग भी…
4    कब्झा हा कन्नड चित्रपट लवकरच मराठी मध्ये डब्ब होऊन प्रदर्शित होणार आहे जर ह्या चित्रपटाला चांगला प्रतिसाद मिळाला तर आप…
Name: tweet, dtype: object


**Remove stopwords**

In [ ]:
!pip install stopwordsiso

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.2 MB/s eta 0:00:00


In [ ]:
import stopwordsiso
stopwordsiso.has_lang("mr")  # check if there is a stopwords for Thai language
from stopwordsiso import stopwords

# Load Marathi stopwords
marathi_stopwords = stopwords("mr")

# Function to remove stopwords
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in marathi_stopwords]
    return ' '.join(filtered_words)

# Apply the remove_stopwords function to the 'tweet' column
df['without_stopwords'] = df['tweet'].apply(remove_stopwords)

# Display the DataFrame with cleaned tweets
print(df[['tweet', 'without_stopwords','subtask_a']].head())

                                                                                                                     tweet  \
0                 आजच्या जनता दरबारात जळगाव जिल्ह्यातील चाळीसगावचे रहिवासी माजी सैनिक सोनू महाजन आले होते भाजपचे तत्कालीन…   
1                       कुणी कविता करत असतं तर कुणी कविता जगत असतं कुणी कविता वाचत असतं तर कुणाला कविताच वाचवत असते पल्लवी   
2        आम्हाला इतिहासातील औरंगजेबशी काही घेणे नाही आमच्या कडे आमचा बेकायदेशीर रित्या आलेला हक्काचा औरंगजेब आहे तो एका क…   
3                            गँभीर प्रकरण महाराष्ट्राची अवस्था बिकट आहे भाषणात मोठे शब्द वापरणे ऐकले कृती करावी उद्योग भी…   
4  कब्झा हा कन्नड चित्रपट लवकरच मराठी मध्ये डब्ब होऊन प्रदर्शित होणार आहे जर ह्या चित्रपटाला चांगला प्रतिसाद मिळाला तर आप…   

                                                                                         without_stopwords  \
0          आजच्या जनता दरबारात जळगाव जिल्ह्यातील चाळीसगावचे रहिवासी माजी सैनिक सोनू महाजन भाजपचे तत्कालीन…   
1             कुणी कवित

In [ ]:
print(df[df['subtask_a']=='Offensive'].head())

Empty DataFrame
Columns: [tweet, subtask_a, without_stopwords]
Index: []


**HASOC Dataset**

In [ ]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  # Show all columns
df2 = pd.read_excel('/kaggle/input/marathi-hasoc-dataset/mr_Hasoc2021_train.xlsx')

# Display the first few rows of the dataset
print(df2.head())


      text_id  \
0  hasoc_mr_1   
1  hasoc_mr_2   
2  hasoc_mr_3   
3  hasoc_mr_4   
4  hasoc_mr_5   

                                                                                                                  text  \
0     भारत 15 ऑगस्ट 1947 ला स्वतंत्र झाला आणि त्यानंतर तब्बल 2.5 वर्षांनी म्हणजे 26 जानेवारी 1950 साली भारताला राज्यघ…   
1  स्वत ला हवा तसा बाइट किंवा प्रतिक्रिया घेण्यासाठी ॲन्करबाई किती हट्टाला पेटलीय. जोरजबरदस्तीने तोंडात वाक्य घुसवतेय.   
2                           5 व्या नंबरची अर्थव्यवस्था आहे भारताची जगात 2014 पर्यंत.... चंप्या  आता पण 5 वर आहे... अण…   
3                                                                        च्यायला म्हणजे दुबईचा फोन ही पुडीच निघाली की.   
4                ह्याला खरंतर कधीच आत टाकला पाहिजे होता. पैसा आणि स्टारडमच्या जोरावर चाललाय सगळं. एकदा ह्याच्यावर घाल…   

  task_1  
0    NOT  
1    NOT  
2    NOT  
3    HOF  
4    HOF  


**Label Encoding**

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'task_1' column
df2['task_1'] = label_encoder.fit_transform(df2['task_1'])
print(df2.head())  #Hate means 0

      text_id  \
0  hasoc_mr_1   
1  hasoc_mr_2   
2  hasoc_mr_3   
3  hasoc_mr_4   
4  hasoc_mr_5   

                                                                                                                  text  \
0     भारत 15 ऑगस्ट 1947 ला स्वतंत्र झाला आणि त्यानंतर तब्बल 2.5 वर्षांनी म्हणजे 26 जानेवारी 1950 साली भारताला राज्यघ…   
1  स्वत ला हवा तसा बाइट किंवा प्रतिक्रिया घेण्यासाठी ॲन्करबाई किती हट्टाला पेटलीय. जोरजबरदस्तीने तोंडात वाक्य घुसवतेय.   
2                           5 व्या नंबरची अर्थव्यवस्था आहे भारताची जगात 2014 पर्यंत.... चंप्या  आता पण 5 वर आहे... अण…   
3                                                                        च्यायला म्हणजे दुबईचा फोन ही पुडीच निघाली की.   
4                ह्याला खरंतर कधीच आत टाकला पाहिजे होता. पैसा आणि स्टारडमच्या जोरावर चाललाय सगळं. एकदा ह्याच्यावर घाल…   

   task_1  
0       1  
1       1  
2       1  
3       0  
4       0  


In [ ]:
print(df2[df2['task_1']==0].head())

        text_id  \
3    hasoc_mr_4   
4    hasoc_mr_5   
5    hasoc_mr_6   
7    hasoc_mr_8   
10  hasoc_mr_11   

                                                                                                     text  \
3                                                           च्यायला म्हणजे दुबईचा फोन ही पुडीच निघाली की.   
4   ह्याला खरंतर कधीच आत टाकला पाहिजे होता. पैसा आणि स्टारडमच्या जोरावर चाललाय सगळं. एकदा ह्याच्यावर घाल…   
5                          भोसडीच्या तुझ्या आईचा मी नवरा आहे. तुझ्या आईची गांड बघ. मागून गांड मारून मारू…   
7   तुला कुत्र तर विचारत का ? आणि हो एकट्या कंगणाने तुमच्या गोट्या चोळून पार धूर काढला  घंटा काय करावं क…   
10         राम कदम वागण्यात नाही तर बोलण्यात चुकला  बीजेपी मग अजितदादा कुठे तुमच्या तोंडात मूतले होते का?   

    task_1  
3        0  
4        0  
5        0  
7        0  
10       0  


**Convert Emojis to text**

In [ ]:
!pip install emoji



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 8.5 MB/s eta 0:00:00


In [ ]:
import emoji

# Function to demojize text
def demojize_text(text):
    return emoji.demojize(text)

# Apply the function to the 'text' column
df2['text'] = df2['text'].apply(demojize_text)


print(df2['text'].head())
print(df2.loc[1320, 'text'])

0       भारत 15 ऑगस्ट 1947 ला स्वतंत्र झाला आणि त्यानंतर तब्बल 2.5 वर्षांनी म्हणजे 26 जानेवारी 1950 साली भारताला राज्यघ…
1    स्वत ला हवा तसा बाइट किंवा प्रतिक्रिया घेण्यासाठी ॲन्करबाई किती हट्टाला पेटलीय. जोरजबरदस्तीने तोंडात वाक्य घुसवतेय.
2                             5 व्या नंबरची अर्थव्यवस्था आहे भारताची जगात 2014 पर्यंत.... चंप्या  आता पण 5 वर आहे... अण…
3                                                                          च्यायला म्हणजे दुबईचा फोन ही पुडीच निघाली की.
4                  ह्याला खरंतर कधीच आत टाकला पाहिजे होता. पैसा आणि स्टारडमच्या जोरावर चाललाय सगळं. एकदा ह्याच्यावर घाल…
Name: text, dtype: object
नविचारता माझाचाटतो वर मलाच बोलतो:face_with_hand_over_mouth: झाट वाढली आहे येतो का उपटायला 40पैसे देतो... मी खाउन…


**Remove unwanted patterns in text**

In [ ]:
import re

pattern = r'(\d+)\sसाली'

# Apply the regular expression to the 'text' column
df2['text'] = df2['text'].apply(lambda x: re.sub(pattern, '', x))

# Assuming 'tweet' is the column containing text with special characters, numbers, and the word 'user'
df2['text'] = df2['text'].str.replace('[\d@?()~|+।‘•©-]', '', regex=True)

pattern = r'\.+'

# Apply the regular expression to the 'text' column
df2['text'] = df2['text'].apply(lambda x: re.sub(pattern, '', x))

df2['text'] = df2['text'].apply(lambda x: re.sub(r'\s+', ' ', x))


# Display the modified DataFrame
print(df2['text'].head())

0                              भारत ऑगस्ट ला स्वतंत्र झाला आणि त्यानंतर तब्बल वर्षांनी म्हणजे जानेवारी भारताला राज्यघ…
1    स्वत ला हवा तसा बाइट किंवा प्रतिक्रिया घेण्यासाठी ॲन्करबाई किती हट्टाला पेटलीय जोरजबरदस्तीने तोंडात वाक्य घुसवतेय
2                                            व्या नंबरची अर्थव्यवस्था आहे भारताची जगात पर्यंत चंप्या आता पण वर आहे अण…
3                                                                         च्यायला म्हणजे दुबईचा फोन ही पुडीच निघाली की
4                  ह्याला खरंतर कधीच आत टाकला पाहिजे होता पैसा आणि स्टारडमच्या जोरावर चाललाय सगळं एकदा ह्याच्यावर घाल…
Name: text, dtype: object


**Remove punctuation marks**

In [ ]:
import string
# Function to remove punctuations from Marathi text
def remove_punctuations(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# Apply the function to the 'tweet' column
df2['text'] = df2['text'].apply(remove_punctuations)
print(df2['text'].head())

0                              भारत ऑगस्ट ला स्वतंत्र झाला आणि त्यानंतर तब्बल वर्षांनी म्हणजे जानेवारी भारताला राज्यघ…
1    स्वत ला हवा तसा बाइट किंवा प्रतिक्रिया घेण्यासाठी ॲन्करबाई किती हट्टाला पेटलीय जोरजबरदस्तीने तोंडात वाक्य घुसवतेय
2                                            व्या नंबरची अर्थव्यवस्था आहे भारताची जगात पर्यंत चंप्या आता पण वर आहे अण…
3                                                                         च्यायला म्हणजे दुबईचा फोन ही पुडीच निघाली की
4                  ह्याला खरंतर कधीच आत टाकला पाहिजे होता पैसा आणि स्टारडमच्या जोरावर चाललाय सगळं एकदा ह्याच्यावर घाल…
Name: text, dtype: object


**Remove stop words**

In [ ]:
import stopwordsiso
stopwordsiso.has_lang("mr")
from stopwordsiso import stopwords

# Load Marathi stopwords
marathi_stopwords = stopwords("mr")

# Function to remove stopwords
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in marathi_stopwords]
    return ' '.join(filtered_words)

# Apply the remove_stopwords function to the 'tweet' column
df2['without_stopwords'] = df2['text'].apply(remove_stopwords)

# Display the DataFrame with cleaned tweets
print(df2[['text', 'without_stopwords']].head())

                                                                                                                text  \
0                            भारत ऑगस्ट ला स्वतंत्र झाला आणि त्यानंतर तब्बल वर्षांनी म्हणजे जानेवारी भारताला राज्यघ…   
1  स्वत ला हवा तसा बाइट किंवा प्रतिक्रिया घेण्यासाठी ॲन्करबाई किती हट्टाला पेटलीय जोरजबरदस्तीने तोंडात वाक्य घुसवतेय   
2                                          व्या नंबरची अर्थव्यवस्था आहे भारताची जगात पर्यंत चंप्या आता पण वर आहे अण…   
3                                                                       च्यायला म्हणजे दुबईचा फोन ही पुडीच निघाली की   
4                ह्याला खरंतर कधीच आत टाकला पाहिजे होता पैसा आणि स्टारडमच्या जोरावर चाललाय सगळं एकदा ह्याच्यावर घाल…   

                                                                                                   without_stopwords  
0                                            भारत ऑगस्ट ला स्वतंत्र त्यानंतर तब्बल वर्षांनी जानेवारी भारताला राज्यघ…  
1  स्वत ला हवा तसा बाइट किंवा प्रतिक्रिया

In [ ]:
df['tweet']=df['without_stopwords']
df2['text']=df2['without_stopwords']

In [ ]:
print(df.columns)
print(df2.columns)

Index(['tweet', 'subtask_a', 'without_stopwords'], dtype='object')
Index(['text_id', 'text', 'task_1', 'without_stopwords'], dtype='object')


**Remove unwanted columns**

In [ ]:
# Specify the columns to remove
columns_to_remove = ['without_stopwords']

# Remove columns that are in the specified list
df.drop(columns=columns_to_remove, inplace=True)



In [ ]:
columns_to_remove2 = ['text_id','without_stopwords']

# Remove columns that are in the specified list
df2.drop(columns=columns_to_remove2, inplace=True)


In [ ]:
df = df.rename(columns={'subtask_a': 'label'})
df2 = df2.rename(columns={'text': 'tweet','task_1': 'label'})

**Merge datasets**

In [ ]:
# Concatenate DataFrames along rows
concatenated_df = pd.concat([df, df2], ignore_index=True)

In [ ]:
concatenated_df.columns


Index(['tweet', 'label'], dtype='object')

In [ ]:
print(df.shape)
print(df2.shape)
print(concatenated_df.shape)

(3102, 2)
(1874, 2)
(4976, 2)


**Remove duplicate rows**

In [ ]:
concatenated_df.drop_duplicates(inplace=True)

****Tokenization****

In [ ]:
!pip install transformers



In [ ]:
import pandas as pd
from transformers import BertTokenizer

# Load mBERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

def tokenize_long_text(text, max_length=512, stride=100):
    tokens = []
    for i in range(0, len(text), stride):
        chunk = text[i:i+max_length]
        chunk_tokens = tokenizer.encode(chunk, add_special_tokens=True)
        tokens.extend(chunk_tokens)
    return tokens


# Tokenize the 'tweet' column with sliding window
concatenated_df['tokenized_tweet'] = concatenated_df['tweet'].apply(lambda x: tokenize_long_text(x, max_length=512, stride=100))

# Convert token IDs to tokens
concatenated_df['tokenized_tweet_tokens'] = concatenated_df['tokenized_tweet'].apply(lambda x: tokenizer.convert_ids_to_tokens(x))

# Display the DataFrame with tokenized tweets
print(concatenated_df.head())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

                                                                                                     tweet  \
0          आजच्या जनता दरबारात जळगाव जिल्ह्यातील चाळीसगावचे रहिवासी माजी सैनिक सोनू महाजन भाजपचे तत्कालीन…   
1             कुणी कविता करत असतं कुणी कविता जगत असतं कुणी कविता वाचत असतं कुणाला कविताच वाचवत असते पल्लवी   
2             आम्हाला इतिहासातील औरंगजेबशी घेणे आमच्या कडे आमचा बेकायदेशीर रित्या आलेला हक्काचा औरंगजेब क…   
3                गँभीर प्रकरण महाराष्ट्राची अवस्था बिकट भाषणात मोठे शब्द वापरणे ऐकले कृती करावी उद्योग भी…   
4  कब्झा कन्नड चित्रपट लवकरच मराठी मध्ये डब्ब होऊन प्रदर्शित जर ह्या चित्रपटाला चांगला प्रतिसाद मिळाला आप…   

   label  \
0      1   
1      1   
2      1   
3      1   
4      1   

                                                                                                                                                                                                                                                                          

**Sentence Embedding**

In [ ]:
!pip install fasttext


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199773 sha256=00c4ac4eabc088ba2f2a431f44b69c56601ef43dc2513caabae0e9fb33bef8bf
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import os
import fasttext.util

# Specify the path where you want to save the model
model_path = '/content/cc.mr.300.bin.gz.part'

# Download the pre-trained FastText model for Marathi
fasttext.util.download_model('mr', if_exists='ignore')

# Move the downloaded model to the specified path
os.rename('cc.mr.300.bin', model_path)

# Load the pre-trained model
model = fasttext.load_model(model_path)





In [ ]:
# Function to get the sentence vector using FastText
def get_sentence_vector(text):
    return model.get_sentence_vector(text)

# Apply the function to the 'tweet' column to get sentence vectors
concatenated_df['tweet_vector'] = concatenated_df['tweet'].apply(get_sentence_vector)


In [ ]:
print(concatenated_df.head())

                                                                                                     tweet  \
0          आजच्या जनता दरबारात जळगाव जिल्ह्यातील चाळीसगावचे रहिवासी माजी सैनिक सोनू महाजन भाजपचे तत्कालीन…   
1             कुणी कविता करत असतं कुणी कविता जगत असतं कुणी कविता वाचत असतं कुणाला कविताच वाचवत असते पल्लवी   
2             आम्हाला इतिहासातील औरंगजेबशी घेणे आमच्या कडे आमचा बेकायदेशीर रित्या आलेला हक्काचा औरंगजेब क…   
3                गँभीर प्रकरण महाराष्ट्राची अवस्था बिकट भाषणात मोठे शब्द वापरणे ऐकले कृती करावी उद्योग भी…   
4  कब्झा कन्नड चित्रपट लवकरच मराठी मध्ये डब्ब होऊन प्रदर्शित जर ह्या चित्रपटाला चांगला प्रतिसाद मिळाला आप…   

   label  \
0      1   
1      1   
2      1   
3      1   
4      1   

                                                                                                                                                                                                                                                                          

In [ ]:
print(concatenated_df.columns)

Index(['tweet', 'label', 'tokenized_tweet', 'tokenized_tweet_tokens',
       'tweet_vector'],
      dtype='object')


**Convert to csv**

In [ ]:
concatenated_df.to_csv('marathi_preprocessed.csv', index=False)
